In [1]:
#Importing Basic Python Libraries
import numpy as np
import pandas as pd
import scipy as ss
import openpyxl as opxl
from pyomo.environ import *
import matplotlib.pyplot as plt

In [2]:
#A function to index the names
##Inputs to the function are-(DataFrame name,Column Index number,Index Label)
###Column Index number, eg. 0 for first column, 1 for second column and so on
###Index Label which is to added to index the elements
 
def index_set(dataframe,column_index,index_label):
  x = []
  for i in range(0,len(dataframe.iloc[:,0])):
    x.append((dataframe.iloc[i,column_index]+str(index_label)).upper())
    i=i+1
  return x

In [3]:
#A Function to create a dataframe by combining columns from excel workbook
##Inputs required are - (Name of DataFrame, Number of columns to combine)
#Note:: Number of columns input can take only 2 or 3 as an input
##Press 2 if he dictionary is of the form {Key: Value}
##Press 3 if the dictionary is of the form {(Key1,key2): Value}
 
def to_dictionary(dataframe, num_of_columns):
  x = {}
  if num_of_columns == 3:
    for i in range(0,len(dataframe.iloc[:,0])):
      x.update({(dataframe.iloc[i,0],dataframe.iloc[i,1]):\
                dataframe.iloc[i,2]})
  elif num_of_columns ==2:
    for i in range(0,len(dataframe.iloc[:,0])):
      x.update({dataframe.iloc[i,0]:dataframe.iloc[i,1]})
  else:
    print('This function only takes input for num_of_columns as 2 or 3')
  return x

Importing **Data_File.xlsx** workbook containing all the data pertainent to the
logistics network. The worksheet names along with the descripton of the data they contain are mentioned in text above each code line

In [4]:
#Excel workbook
wb = opxl.load_workbook('Data_File .xlsx',data_only = True)

##### **Sheet Name** - ***Customers***
**Data** - *Contains names of customers along with their location details (latitude, longitude and state identifier).*

In [5]:
#Customers Information DataFrame
##Customer Name as DataFrame Row Index
##State Identifier, Latitude and Longitude as the column heading of DataFrame 
 
customers_info = pd.DataFrame(wb['Customers'].values)
customers_info.columns = customers_info.iloc[0,:]
customers_info.drop([0], inplace = True)

In [6]:
# Reading Cuustomer names as a python list
# Note:: All the customer names are indexed, by adding ".C" in the end.
 
customers = index_set(customers_info, 0, '.C')
len(customers)

332

##### **Sheet Name** - ***EPA***
**Data** - *Contains names, Dispatch Capacity, Road and Rail handling costs and inventory holding costs of External Processing Agents.*

In [7]:
#EPA names
##Python list containing the names
## Note:: All the mfg. EPA names are indexed by adding ".E" in the end.
 
epa_info = pd.DataFrame(wb['EPA'].values)
epa_info.columns = epa_info.iloc[0,:]
epa_info.drop([0],axis=0, inplace=True)
epa = epa_info.copy()
epa = index_set(epa,0,'.E')
len(epa)

76

In [8]:
#EPA dispatch capacity 
##Python Dictionary with keys and values as -
###Key - (EPA Name)
###Value - (Dispatch capacity in tonns at EPA)
 
epa_cap = epa_info.copy()
epa_cap.iloc[:,0] = index_set(epa_cap,0,'.e')
#Total Dicpatch capacity of EPA
epa_cap.drop(['Latitude','Longitude'],axis=1,inplace=True)
epa_disp_cap = to_dictionary(epa_cap,2)
epa_rail_disp_cap = epa_cap.drop(['Total Despatch Capacity (t)'], axis = 1)
#Rail Dispatch capacity of EPA
epa_rail_disp_cap = to_dictionary(epa_rail_disp_cap,2)
print(len(epa_disp_cap),len(epa_rail_disp_cap))

76 76


In [9]:
#EPA Fixed and variable handling costs for road transport
##Python Dictionary with keys and values as -
###Key - (EPA Name)
###Value - (Variable/Fixed habdling cost for road transported goods at EPA)
 
epa_hndlng_road = epa_info.copy()
epa_hndlng_road.drop(['Total Despatch Capacity (t)', 'Rail Capacity (t)',\
                     'Latitude','Longitude'],axis=1,inplace=True)
epa_hndlng_road.iloc[:,0] = index_set(epa_hndlng_road,0,'.e')
epa_hndlng_vc_road = epa_hndlng_road.drop(['Fixed Handling Cost Road'],axis=1)
epa_hndlng_vc_road = to_dictionary(epa_hndlng_vc_road,2)
epa_hndlng_fc_road = epa_hndlng_road.drop(['Variable Handling Cost Road'],\
                                          axis=1)
epa_hndlng_fc_road = to_dictionary(epa_hndlng_fc_road,2)
print(len(epa_hndlng_fc_road),len(epa_hndlng_vc_road))

76 76


In [10]:
#EPA Fixed and variable handling costs for rail transport
##Python Dictionary with keys and values as -
###Key - (EPA Name)
###Value - (Variable/Fixed habdling cost for rail transported goods at EPA)
 
epa_hndlng_rail = epa_info.copy()
epa_hndlng_rail.drop(['Total Despatch Capacity (t)', 'Rail Capacity (t)',\
                      'Fixed Handling Cost Road','Variable Handling Cost Road',\
                     'Latitude','Longitude'],axis=1,inplace=True)
epa_hndlng_rail.iloc[:,0] = index_set(epa_hndlng_rail,0,'.e')
epa_hndlng_vc_rail = epa_hndlng_rail.drop(['Fixed Handling Cost Rail'],axis=1)
epa_hndlng_vc_rail = to_dictionary(epa_hndlng_vc_rail,2)
epa_hndlng_fc_rail = epa_hndlng_rail.drop(['Variable Handling Cost Rail'],\
                                          axis=1)
epa_hndlng_fc_rail = to_dictionary(epa_hndlng_fc_rail,2)
print(len(epa_hndlng_fc_rail),len(epa_hndlng_vc_rail))

76 76


In [11]:
#EPA Inventory Holding Cost
##Python Dictionary with keys and values as -
###Key - (Stockyard Name)
###Value - (Inventory Holding Factor at EPA)
 
epa_inv_hf = epa_info.copy()
epa_inv_hf.drop(['Total Despatch Capacity (t)', 'Rail Capacity (t)',\
                 'Fixed Handling Cost Road','Variable Handling Cost Road',\
                 'Fixed Handling Cost Rail','Variable Handling Cost Rail',\
                'Latitude','Longitude'],axis=1,inplace=True)
epa_inv_hf.iloc[:,0]=index_set(epa_inv_hf,0,'.e')
epa_inv_hf = to_dictionary(epa_inv_hf,2)
len(epa_inv_hf)

76

##### **Sheet Name** - ***SYD***
**Data** - *Contains names, Dispatch Capacity, Road and Rail handling costs and inventory holding costs of Stockyards.*


In [12]:
#Stockyard Names
##Python list containing Stockyard Names
## Note:: All the Stockyard names are indexed by adding ".S" in the end.
 
syd_info = pd.DataFrame(wb['SYD'].values)
syd_info.columns = syd_info.iloc[0,:]
syd_info.drop([0], axis=0, inplace=True)
syd = index_set(syd_info, 0,'.S')
len(syd)

76

In [13]:
#SYD throughput capacity 
##Python Dictionary with keys and values as -
###Key - (SYD Name)
###Value - (Throughput capacity in tonns at Stockyard)
 
syd_cap = syd_info.copy()
syd_cap.iloc[:,0] = index_set(syd_cap,0,'.s')
syd_cap.drop(['Latitude','Longitude'],axis=1,inplace=True)
syd_total_thput = syd_cap.drop(['Rail Capacity (t)'],axis=1)
syd_total_thput = to_dictionary(syd_total_thput,2)###Total throughput capacity of SYD
syd_rail_thput = syd_cap.drop(['Maximum Throughput (t)'],axis=1)
syd_rail_thput = to_dictionary(syd_rail_thput,2)###Rail throughput capacity of SYD
print(len(syd_total_thput),len(syd_rail_thput))

76 76


In [14]:
#Stockyard sliding costs
##Python Dictionary with keys and values as -
###Key - (Stockyard Name)
###Value - (Sliding cost per tonn of goods at SYD)
syd_sldng_cost = syd_info.copy()
syd_sldng_cost.drop(['Rail Capacity (t)','Maximum Throughput (t)',\
                    'Latitude','Longitude'],axis=1, inplace=True)
syd_sldng_cost.iloc[:,0]=index_set(syd_sldng_cost,0,'.s')
syd_sldng_cost = to_dictionary(syd_sldng_cost,2)
len(syd_sldng_cost)

76

In [15]:
#Stockyard Fixed and variable handling costs for road transport
##Python dictionary with keys and values as -
###Key - (Stockyard Name)
###Value-(Variable/Fixed habdling cost per tonn of road transported goods at SYD)
 
syd_hndlng_road = syd_info.copy()
syd_hndlng_road.drop(['Rail Capacity (t)','Maximum Throughput (t)',\
                      'Siding Handling Cost (Rs/t)','Latitude','Longitude'],\
                     axis=1, inplace=True)
syd_hndlng_road.iloc[:,0] = index_set(syd_hndlng_road,0,'.s')
syd_hndlng_vc_road = syd_hndlng_road.drop(['Fixed Handling Cost Road'],axis=1)
syd_hndlng_vc_road = to_dictionary(syd_hndlng_vc_road,2)
syd_hndlng_fc_road = syd_hndlng_road.drop(['Variable Handling Cost Road'],\
                                          axis=1)
syd_hndlng_fc_road = to_dictionary(syd_hndlng_fc_road,2)
print(len(syd_hndlng_vc_road),len(syd_hndlng_fc_road))

76 76


In [16]:
#Stockyard Fixed and variable handling costs for road transport
##Python dictionary with keys and values as -
###Key - (Stockyard Name)
###Value-(Variable/Fixed habdling cost per tonn of road transported goods at SYD)

syd_hndlng_rail = syd_info.copy()
syd_hndlng_rail.drop(['Rail Capacity (t)','Maximum Throughput (t)',\
                      'Siding Handling Cost (Rs/t)','Fixed Handling Cost Road'\
                      ,'Variable Handling Cost Road','Latitude','Longitude'],\
                     axis=1, inplace=True)
syd_hndlng_rail.iloc[:,0] = index_set(syd_hndlng_rail,0,'.s')
syd_hndlng_vc_rail = syd_hndlng_rail.drop(['Fixed Handling Cost Rail'],axis=1)
syd_hndlng_vc_rail = to_dictionary(syd_hndlng_vc_rail,2)
syd_hndlng_fc_rail = syd_hndlng_rail.drop(['Variable Handling Cost Rail'],\
                                          axis=1)
syd_hndlng_fc_rail = to_dictionary(syd_hndlng_fc_rail,2)
print(len(syd_hndlng_vc_rail),len(syd_hndlng_fc_rail))

76 76


In [17]:
#Stockyard Inventory Holding Cost
##Python Dictionary with keys and values as -
###Key - (Stockyard Name)
###Value - (Inventory Holding Factor at SYD)

syd_inv_hf = syd_info.copy()
syd_inv_hf.drop(['Rail Capacity (t)','Maximum Throughput (t)',\
                 'Siding Handling Cost (Rs/t)','Fixed Handling Cost Road',\
                 'Variable Handling Cost Road', 'Fixed Handling Cost Rail'\
                 ,'Variable Handling Cost Rail','Latitude','Longitude'],\
                axis=1, inplace=True)
syd_inv_hf.iloc[:,0]=index_set(syd_inv_hf,0,'.s')
syd_inv_hf = to_dictionary(syd_inv_hf,2)
len(syd_inv_hf)

76

##### **Sheet Name** - ***Sending_Ports***
**Data** - *Contains names of all the ports along with their location details(latitude, longitude and state identifier), which are used as sending or dispatch ports.*

In [18]:
#Sending Ports
##Python list containing Names of Ports used for dispatching goods
## Note::All the names are indexed as Sending Ports by adding ".PO_D" in the end.

disp_ports_info = pd.DataFrame(wb['Sending_Ports'].values)
disp_ports_info.columns = disp_ports_info.iloc[0,:]
disp_ports_info.drop([0], axis=0, inplace=True)
disp_ports_info.iloc[:,0] = index_set(disp_ports_info,0,'.po_d')
disp_ports = list(disp_ports_info.iloc[:,0])
len(disp_ports)

4

In [19]:
#Dispatch Ports Variable Handling Cost 
##Python Dictionary with keys and values as -
###Key - (Port_Name)
###Value - (Variable Handling Cost at each port)

disp_ports_hndlng_cost = disp_ports_info.copy()
disp_ports_hndlng_cost.drop(['State Identifier','Lat','Lon'],axis= 1, inplace=True)
disp_ports_hndlng_cost = to_dictionary(disp_ports_hndlng_cost,2)

In [20]:
#Ports Capacity
##Python Dictionary with keys and values as -
###Key - (Port Name)
###Value - (Min and Max handling capacity at Port)
disp_cap = disp_ports_info.copy()
disp_cap.drop(['State Identifier','Lat','Lon','Variable Handling Charges (Rs/t)'],\
              axis= 1,inplace=True)
disp_port_min_cap = disp_cap.drop(['Maximum Throughput (t)'],axis=1)
disp_port_min_cap = to_dictionary(disp_port_min_cap,2)
disp_port_max_cap = disp_cap.drop(['Minimum Throughput (t)'],axis=1) ###Minimum handling capacity
disp_port_max_cap = to_dictionary(disp_port_max_cap,2) ###Maximum handling capacity

##### **Sheet Name** - ***Receiving_Ports***
**Data** - *Contains names of all the ports along with their location details(latitude, longitude and state identifier), which are used for receiving goods that are dispatched from Sending ports.*

In [21]:
#Receiving Ports
##Python list containing Names of Ports that 'receive goods' from 'disp_ports'
##Note::All the names are indexed as Receiving Ports by adding ".PO_R" in the end

rec_ports_info = pd.DataFrame(wb['Receiving_Ports'].values)
rec_ports_info.columns = rec_ports_info.iloc[0,:]
rec_ports_info.drop([0], axis=0, inplace=True)
rec_ports_info.iloc[:,0] = index_set(rec_ports_info,0,'.po_r')
rec_ports = list(rec_ports_info.iloc[:,0])
len(rec_ports)

8

In [22]:
#Receivable Ports Variable Handling Cost 
##Python Dictionary with keys and values as -
###Key - (Port_Name)
###Value - (Variable Handling Cost at each port)
rec_port_hndlng_cost = rec_ports_info.copy()
rec_port_hndlng_cost.drop(['State Identifier','Lat','Lon'],axis=1, inplace=True)
rec_port_hndlng_cost = to_dictionary(rec_port_hndlng_cost, 2)

In [23]:
#Ports Capacity
##Python Dictionary with keys and values as -
###Key - (Port Name)
###Value - (Min and Max handling capacity at Port)
rec_port_cap = rec_ports_info.copy()
rec_port_cap.drop(['State Identifier','Lat','Lon','Variable Handling Cost']\
                  ,axis=1,inplace=True)
rec_port_min_cap = rec_port_cap.drop(['Maximum Throughput (t)'],axis=1)
rec_port_min_cap = to_dictionary(rec_port_min_cap,2)
rec_port_max_cap = rec_port_cap.drop(['Minimum Throughput (t)'],axis=1) ###Minimum handling capacity
rec_port_max_cap = to_dictionary(rec_port_max_cap,2) ###Maximum handling capacity

##### **Sheet Name** - ***Products***
**Data** - *Contains names of products and their catagorization,i.e. product types, flat products\
long products and total end products present in the network.*

In [24]:
#Products
##Python list containing names of products
products = pd.DataFrame(wb['Products'].values)
product_catagories = list(products.iloc[1:,0].dropna(axis=0))
end_products = list(products.iloc[1:,2].dropna(axis=0))
flat_products = list(products.iloc[1:,4].dropna(axis=0))
long_products = list(products.iloc[1:,6].dropna(axis=0))
print(len(product_catagories),len(end_products),len(flat_products),\
      len(long_products))

9 49 33 16


##### **Sheet Name** - ***Supply***
**Data** - *Contains product wise supply capacity for EPAs.*

In [25]:
#Supply Capacity of EPAs
##Python Dictionary with keys and values as - 
###Key - (Source, Product) 
###Value - (supply capacity of product at source)

supply_epa = pd.DataFrame(wb['Supply'].values)
supply_epa.columns = supply_epa.iloc[0,:]
supply_epa.drop([0],axis=0, inplace=True)
supply_epa.iloc[:,0]=index_set(supply_epa,0,'.e')
supply_epa.set_index("EPA Name", inplace =True, drop = True)
supply_epa = supply_epa.stack().to_dict()
len(supply_epa)

3724

##### **Sheet Name** - ***Supply Mills***
**Data** - *Contains product wise supply capacity information for each mill.*

In [26]:
#Supply Capacity of Mills
##Python Dictionary with keys and values as - 
###Key - (Mill Name, Product) 
###Value - (supply capacity of product at Mill)

supply_mills = pd.DataFrame(wb['Supply Mills'].values)
supply_mills.columns = supply_mills.iloc[0,:]
supply_mills.drop([0],axis=0, inplace=True)
mills = list(supply_mills.iloc[:,0])
supply_mills.set_index("Mills Name", inplace =True, drop = True)
supply_mills = supply_mills*1000
supply_mills = supply_mills.stack().to_dict()
len(supply_mills)

18

##### **Sheet Name** - ***Demand***
**Data** - *Contains product wise demand at all customer nodes.*

In [27]:
#Demand information
##Python Dictionary with keys and values as -
###Key - (Sink, Product) 
###Value - (demand requirement of product at sink)

demand_df = pd.DataFrame(wb['Demand'].values)
demand_df.columns = demand_df.iloc[0,:]
demand_df.drop([0],axis=0, inplace=True)
demand_df.iloc[:,0]=index_set(demand_df,0,'.c')
demand_df.set_index(['Cust. Location'], inplace=True, drop=True)
demand = demand_df.stack().to_dict()
len(demand)

16268

##### **Sheet Name** - ***Rail_Cost***
**Data** - *Contains per tonn cost of transporting goods between two network nodes using railways.\
**Note::** Railways can transport good only between source and transshipment nodes.*

In [28]:
#Rail transport cost between two nodes
rail_cost_sheet = pd.DataFrame(wb['Rail_Cost'].values)
rail_cost_pl_pl = rail_cost_sheet.iloc[:,4:7].dropna(axis=0)
rail_cost_pl_pl.columns = rail_cost_pl_pl.iloc[0,:]
rail_cost_pl_pl.drop([0],axis=0, inplace=True)


In [29]:
#Per tonn rail trans. cost from Mills to EPAs
##Python dictionary with keys and values as -
###Key - (From, To)
###Value - (Unit cost of transportation between nodes)
rail_cost_m_e = rail_cost_sheet.iloc[:,0:3].dropna(axis=0)
rail_cost_m_e.columns = rail_cost_m_e.iloc[0,:]
rail_cost_m_e.drop([0],axis=0, inplace=True)
rail_cost_m_e.iloc[:,1] = index_set(rail_cost_m_e,1,'.e')
rail_cost_m_e = to_dictionary(rail_cost_m_e,3)
len(rail_cost_m_e)

152

In [30]:
# Per tonn rail trans. cost from Non manufacturing EPAs to Stockyards
##Python dictionary with keys and values as -
###Key - (From, To)
###Value - (Unit cost of transportation between nodes)
rail_cost_e_s=pd.DataFrame(data={'From':index_set(rail_cost_pl_pl,0,'.e'),\
                                 'To': index_set(rail_cost_pl_pl,1,'.s'),\
                                 'Rate/tonn':rail_cost_pl_pl.iloc[:,2]})
rail_cost_e_s = to_dictionary(rail_cost_e_s,3)
len(rail_cost_e_s)

5776

In [31]:
# Per tonn rail trans. Cost from Stockyards to Stockyards
##Python dictionary with keys and values as -
###Key - (From, To)
###Value - (Unit cost of transportation between nodes)
rail_cost_s_s=pd.DataFrame(data={'From':index_set(rail_cost_pl_pl,0,'.s'),\
                                 'To': index_set(rail_cost_pl_pl,1,'.s'),\
                                 'Rate/tonn':rail_cost_pl_pl.iloc[:,2]})
rail_cost_s_s = to_dictionary(rail_cost_s_s,3)
len(rail_cost_s_s)

5776

##### **Sheet Name** - ***Road_Cost***
**Data** - *Contains per tonn cost of transporting goods between two network nodes using roadways.\
**Note::** Roadways can transport good between any two network nodes in the network.*

In [32]:
#Road transport cost between two nodes

road_cost_sheet = pd.DataFrame(wb['Road_Cost'].values)
road_cost_sheet.columns = road_cost_sheet.iloc[0,:]
road_cost_sheet.drop([0],axis=0, inplace= True)
road_cost_pl_pl = road_cost_sheet.iloc[:,0:3].dropna(axis=0)
road_cost_m_e = road_cost_sheet.iloc[:,4:7].dropna(axis=0)
road_cost_pl_c = road_cost_sheet.iloc[:,8:11].dropna(axis=0)

In [33]:
#Per tonn road trans. Cost from Mills to Non mfg. EPAs
##Python dictionart with keys and values as -
###Key - (From, To) 
###Value - (Unit cost of transportation between nodes)
road_cost_m_e =pd.DataFrame(data={'From':road_cost_m_e.iloc[:,0],\
                                  'To': index_set(road_cost_m_e,1,'.e'),\
                                  'Rate/tonn':road_cost_m_e.iloc[:,2]})
road_cost_m_e = to_dictionary(road_cost_m_e,3)
len(road_cost_m_e)

152

In [34]:
#Per tonn road trans. Cost from Non mfg. EPAs and Stockyards
##Python dictionart with keys and values as -
###Key - (From, To) 
###Value - (Unit cost of transportation between nodes)
road_cost_e_s =pd.DataFrame(data={'From':index_set(road_cost_pl_pl,0,'.e'),\
                                  'To': index_set(road_cost_pl_pl,1,'.s'),\
                                  'Rate/tonn':road_cost_pl_pl.iloc[:,2]})
road_cost_e_s = to_dictionary(road_cost_e_s,3)
len(road_cost_e_s)

5776

In [35]:
#Per tonn road trans. Cost Non mfg. EPAs to Customers
##Python dictionart with keys and values as -
###Key - (From, To) 
###Value - (Unit cost of transportation between nodes)
road_cost_e_c =pd.DataFrame(data={'From':index_set(road_cost_pl_c,0,'.e'),\
                                  'To': index_set(road_cost_pl_c,1,'.c'),\
                                  'Rate/tonn':road_cost_pl_c.iloc[:,2]})
road_cost_e_c = to_dictionary(road_cost_e_c,3)
len(road_cost_e_c)

25232

In [36]:
#Per tonn road trans. Cost from Stockyards to Stockyards
##Python dictionart with keys and values as -
###Key - (From, To) 
###Value - (Unit cost of transportation between nodes)
road_cost_s_s =pd.DataFrame(data={'From':index_set(road_cost_pl_pl,0,'.s'),\
                                  'To': index_set(road_cost_pl_pl,1,'.s'),\
                                  'Rate/tonn':road_cost_pl_pl.iloc[:,2]})
road_cost_s_s = to_dictionary(road_cost_s_s,3)
len(road_cost_s_s)

5776

In [37]:
#Per tonn road trans. Cost from Stockyards to Customers
##Python dictionart with keys and values as -
###Key - (From, To) 
###Value - (Unit cost of transportation between nodes)
road_cost_s_c =pd.DataFrame(data={'From':index_set(road_cost_pl_c,0,'.s'),\
                                  'To': index_set(road_cost_pl_c,1,'.c'),\
                                  'Rate/tonn':road_cost_pl_c.iloc[:,2]})
road_cost_s_c = to_dictionary(road_cost_s_c,3)
len(road_cost_s_c)

25232

##### **Sheet Name** - ***Coastal_Cost***
**Data** - *Contains per tonn cost of transporting goods between two network nodes using waterways.

In [38]:
#Coastal transport cost between 'disp_ports' and 'rec_ports', and between 'rec_ports' and customers
coastal_cost_sheet = pd.DataFrame(wb['Coastal_Cost'].values)
coastal_cost_sheet.columns = coastal_cost_sheet.iloc[0,:]
coastal_cost_sheet.drop([0], axis=0, inplace=True)
port_to_cust_cost = coastal_cost_sheet.iloc[:,0:3].dropna(axis=0)
port_to_port_cost = coastal_cost_sheet.iloc[:,5:8].dropna(axis=0)
#Goods are transported to 'disp_ports', via road transport

In [39]:
# Per tonn Coastal trans. cost between dispatch ports and receiving ports
##Python Dictionary with keys and values as -
###Key - (From, To)
#Value - (Unit cost of transportation between two ports/ port and customer)
port_to_port_cost =pd.DataFrame(data={'From':index_set(port_to_port_cost,0,'.po_d'),\
                                      'To': index_set(port_to_port_cost,1,'.po_r'),\
                              'Rate/tonn':port_to_port_cost.iloc[:,2]})
port_to_port_cost = to_dictionary(port_to_port_cost,3)
len(port_to_port_cost)

32

In [40]:
# Per tonn Coastal trans. cost between receiving ports and Customers
##Python Dictionary with keys and values as -
###Key - (From, To)
#Value - (Unit cost of transportation between two ports/ port and customer)
port_to_cust_cost =pd.DataFrame(data={'From':index_set(port_to_cust_cost,0,'.po_r')\
                                      ,'To': index_set(port_to_cust_cost,1,'.c')\
                                      ,'Rate/tonn':port_to_cust_cost.iloc[:,2]})
port_to_cust_cost = to_dictionary(port_to_cust_cost,3)
len(port_to_cust_cost)

2656

##### **Sheet Name** - ***PL_Port_Cost***
**Data** - *Contains per tonn cost of transporting goods from potential location to the dispatch ports using roadways.*

In [41]:
#Cost of transporting goods from Supply Locations to 'disp_ports' via road

pl_port_cost = pd.DataFrame(wb['PL_Port_Cost'].values)
x=pl_port_cost.T
pl_port_cost.iloc[0,:] = index_set(x,0,'.po_d')
pl_port_cost.columns = pl_port_cost.iloc[0,:]
pl_port_cost.drop([0], axis = 0, inplace = True)

In [42]:
#Cost of transporting goods from Non mfg. EPA to 'disp_ports' via road
##Python Dictionary with keys and values as -
###Key - (Supply_loc, Port)
###Value - (Unit cost of transportation between EPA/SYD and ports)

e_port_cost = pl_port_cost.copy()
e_port_cost.iloc[:,0] = index_set(e_port_cost,0,'.e')
e_port_cost.set_index('FROM/TO.PO_D', inplace=True, drop = True)
e_port_cost = e_port_cost.stack().to_dict()
len(e_port_cost)

304

In [43]:
#Cost of transporting goods from Stockyards to 'disp_ports' via road
##Python Dictionary with keys and values as -
###Key - (Supply_loc, Port)
###Value - (Unit cost of transportation between EPA/SYD and ports)

s_port_cost = pl_port_cost.copy()
s_port_cost.iloc[:,0] = index_set(s_port_cost,0,'.s')
s_port_cost.set_index('FROM/TO.PO_D', inplace=True, drop = True)
s_port_cost = s_port_cost.stack().to_dict()
len(s_port_cost)

304

##### **Sheet Name** - ***EPA_Conversion_Cost***
**Data** - *Contains cost of converting goods from semifinished form to finished form at a Non manufacturing EPA on per tonn basis.*

In [44]:
#Non mfg. EPA Conversion cost 
##Python Dictionary with keys and values as -
###Key - (EPA Name, Product)
###Value - (Unit cost of converting products at EPA)

e_cnvrsn_cost = pd.DataFrame(wb['NME_Conversion_Cost'].values)
e_cnvrsn_cost.columns = e_cnvrsn_cost.iloc[0,:]
e_cnvrsn_cost.drop([0],axis=0,inplace=True)
e_cnvrsn_cost.iloc[:,0]= index_set(e_cnvrsn_cost,0,'.e')
e_cnvrsn_cost.set_index('EPA Name',inplace = True, drop= True)
e_cnvrsn_cost = e_cnvrsn_cost.stack().to_dict()
len(e_cnvrsn_cost)

3724

##### **Sheet Name** - ***Distance***
**Data** - *Contains distance, in km, between a potential location node and a customer node.*

In [45]:
#Distance between two potential locations and customers
##Python Dictionary with keys and values as -
###Key - (From Node, To Node)
###Value - (Distance between nodes in KM)

dist = pd.DataFrame(wb['Distance'].values)
dist.columns = dist.iloc[0,:]
dist.drop([0],inplace=True)
dist = to_dictionary(dist,3)
len(dist)

25096

##### **Sheet Name** - ***Direct_Shipment***
**Data** - *Contains Percentage of allowed direct shipments for each customer.*

In [46]:
#Percentage direct shipment for flat and long products for each customer
##Python Dictionary with keys and values as -
###Key - (Customer Name)
###Value - (%of direct shipment allowed)

direct_ship = pd.DataFrame(wb['Direct_Shipment'].values)
direct_ship = direct_ship.iloc[:,0:3]
direct_ship.columns = direct_ship.iloc[0,:]
direct_ship.drop([0],inplace=True)
direct_ship.iloc[:,0] = index_set(direct_ship,0,'.c')
direct_ship_flat = direct_ship.drop(['% FLAT direct'],axis=1)
#Direct shipment allowed for flat products at customer location
direct_ship_flat = to_dictionary(direct_ship_flat,2)
direct_ship_long = direct_ship.drop(['% direct (LONG)'],axis=1)
#Direct shipment allowed for long products at customer location
direct_ship_long = to_dictionary(direct_ship_long,2)
print(len(direct_ship_flat), len(direct_ship_flat))

332 332


##### **Sheet Name** - ***SLA_Shipment***
**Data** - *Contains Percentage of allowed SLA shipments for each customer.*

In [47]:
#Percentage SLA shipment for flat and long products for each customer
##Python Dictionary with keys and values as -
###Key - (Customer Name)
###Value - (%of SLA shipment allowed)
sla_ship = pd.DataFrame(wb['SLA_Shipment'].values)
sla_ship = sla_ship.iloc[:,0:3]
sla_ship.columns = sla_ship.iloc[0,:]
sla_ship.drop([0],inplace=True)
sla_ship.iloc[:,0]= index_set(sla_ship,0,'.c')
sla_ship_flat = sla_ship.drop(['% served through SLA (FLAT)'],axis = 1)
#SLA shipment allowed for flat products at customer location
sla_ship_flat = to_dictionary(sla_ship_flat,2)
sla_ship_long = sla_ship.drop(['% served through SLA (LONG)'],axis = 1)
#SLA shipment allowed for long products at customer location
sla_ship_long = to_dictionary(sla_ship_long,2)
print(len(sla_ship_flat), len(sla_ship_flat))

332 332


##### **Sheet Name** - ***Type_to_Form***
**Data** - *Explains which end products can be produced from each of products supplied by mills.*

In [48]:
#Reaading Type to form conversion principle as a dictionary
##Python Dictionary with keys and values as -
#Key-(Product Type, Product Form)
#Value-(a binary variable-1 if it's possible to convert form product type to form, 0 otherwise)
t2f = pd.DataFrame(wb['Type_to_Form'].values)
t2f.columns = t2f.iloc[0,:]
t2f.drop([0],axis=0,inplace = True)
t2f.set_index('Type/Form', inplace=True, drop = True)
t2f = t2f.stack().to_dict()
len(t2f)

441

# **Formulating the model**

Creating a **Concrete Model**, named as ***model***

In [49]:
model = ConcreteModel()

### **Sets**

In [50]:
#Set Mills
model.M = Set(initialize = mills, doc = 'mills')

In [51]:
#Set EPAs
model.E = Set(initialize = epa, doc = 'EPAs')

In [52]:
#Set Stockyards
model.S = Set(initialize = syd, doc= 'Stockyards')

In [53]:
#Set Customers
model.C = Set(initialize = customers, doc = 'customers')

In [54]:
#Set Products
model.I = Set(initialize = end_products, doc= 'Products')

In [55]:
#Set Long Products
model.LP = Set(initialize = long_products, doc = 'Long Products')

In [56]:
#Set Flat Products
model.FP = Set(initialize = flat_products, doc = 'Flat Products')

In [57]:
#Set Potential Locations
model.IM = Set(initialize = product_catagories, doc = 'Products supplied by  mills')

In [58]:
#Set Dispatch ports
model.PO_D = Set(initialize = disp_ports, doc = 'Sending Ports or Dispatch Ports')

In [59]:
#Set Receivving Ports
model.PO_R = Set(initialize = rec_ports, doc = 'Receiving Ports')

### **Parameters**

##### **Parameter** - **Rail Cost**
Contains per tonn cost of transporting goods between nodes via. railways.

In [60]:
#Rail Transportation cost between all supply locations
##Per tonn Cost of transporting via rail from Mills to EPAs
model.rail_cost_m_e = Param(model.M,model.E,initialize = rail_cost_m_e)
##Per tonn Cost of transporting via rail from EPAs to Stokyards
model.rail_cost_e_s = Param(model.E,model.S,initialize = rail_cost_e_s)
##Per tonn Cost of transporting via rail from Stockyards to Stockyards
model.rail_cost_s_s = Param(model.S,model.S,initialize = rail_cost_s_s)

##### **Parameter** - **Road Cost**
Contains per tonn cost of transporting goods between nodes via roadways.

In [61]:
#Road transportation cost
##Per tonn Cost of transporting via road from Mills to EPAs
model.road_cost_m_e  = Param(model.M, model.E,initialize = road_cost_m_e)
##Per tonn Cost of transporting via road from EPAs to Stokyards
model.road_cost_e_s  = Param(model.E, model.S,initialize = road_cost_e_s)
##Per tonn Cost of transporting via road from EPAs to Customers
model.road_cost_e_c  = Param(model.E, model.C,initialize = road_cost_e_c)
##Per tonn Cost of transporting via road from Stockyards to Stockyards
model.road_cost_s_s  = Param(model.S, model.S,initialize = road_cost_s_s)
##Per tonn Cost of transporting via road from Stockyards to Customers
model.road_cost_s_c  = Param(model.S, model.C,initialize = road_cost_s_c)

##### **Parameter** - **Coastal Cost**
Contains per tonn cost of transporting goods between nodes via waterways.

In [62]:
#Coastal transportation cost
##Per tonn Cost of transporting via waterways from EPAs to Dispatch Ports
model.e_port_cost = Param(model.E,model.PO_D,initialize = e_port_cost)
##Per tonn Cost of transporting via waterways from Stockyards to Dispatch Ports
model.s_port_cost = Param(model.S,model.PO_D,initialize = s_port_cost)
##Per tonn Cost of transporting via waterways from Dispatch Ports to Receiving Ports
model.port_port_cost= Param(model.PO_D,model.PO_R,initialize = port_to_port_cost)
##Per tonn Cost of transporting via waterways from Receiving Ports to customers
model.port_cust_cost= Param(model.PO_R,model.C,initialize = port_to_cust_cost)

##### **Parameter** - **Ports Handling costs**
Contains per tonn cost of Handling goods at ports.


In [63]:
#Ports variable handling cost at Dispatch Ports
model.disp_port_hndlng_cost = Param(model.PO_D,initialize = disp_ports_hndlng_cost)
#Ports variable handling cost at Dispatch Ports
model.rec_port_hndlng_cost = Param(model.PO_R,initialize = rec_port_hndlng_cost)

##### **Parameter** - **EPA Conversion costs**
Contains per tonn cost of Converting model.IM to model.I, at each EPA.


In [64]:
#EPA Conversion costs
model.epa_conversn_cost = Param(model.E,model.I, initialize = e_cnvrsn_cost)

##### **Parameter** - **EPA Variable Handling costs**
Contains per tonn cost of Handling goods at EPAs.

In [65]:
#EPA Variable Handling costs for road
model.epa_hndlng_vc_road = Param(model.E, initialize = epa_hndlng_vc_road)

In [66]:
#EPA fixed Handling costs for road
model.epa_hndlng_fc_road = Param(model.E,initialize = epa_hndlng_fc_road)

In [67]:
#EPA fixed Handling costs for rail
model.epa_hndlng_fc_rail = Param(model.E,initialize = epa_hndlng_fc_rail)

In [68]:
#EPA Variable Handling costs for rail
model.epa_hndlng_vc_rail = Param(model.E,initialize = epa_hndlng_vc_rail)

##### **Parameter** - **SYD Variable Handling costs**
Contains per tonn cost of Handling goods at Stockyards.


In [69]:
#SYD Variable Handling costs for road
model.syd_hndlng_vc_road = Param(model.S,initialize = syd_hndlng_vc_road)

In [70]:
#SYD fixed Handling costs for road
model.syd_hndlng_fc_road = Param(model.S,initialize = syd_hndlng_fc_road)

In [71]:
#SYD variable Handling costs for rail
model.syd_hndlng_vc_rail = Param(model.S,initialize = syd_hndlng_vc_rail)

In [72]:
#SYD fixed Handling costs for rail
model.syd_hndlng_fc_rail = Param(model.S,initialize = syd_hndlng_fc_rail)

In [73]:
#SYD sliding costs
model.syd_sldng_cost = Param(model.S,initialize = syd_sldng_cost)

##### **Parameter** - **Customer Demand**
Contains productwise demand at each customer node 

In [74]:
#Customer demand
model.demand = Param(model.C, model.I, initialize = demand)

##### **Parameter** - **EPA Supply**
Contains productwise supply from each EPA

In [75]:
#EPA Supply
model.supply_epa = Param(model.E, model.I, initialize = supply_epa)

##### **Parameter** - **Mills Supply**
Contains productwise supply from each Mill

In [76]:
#Mills Supply
model.supply_m = Param(model.M, model.IM, initialize = supply_mills)

##### **Parameter** - **Ports Handling Capacity**
Contains min and max port handling capacities

In [77]:
#Dispactch ports min and max capacities
model.disp_port_min_cap = Param(model.PO_D, initialize = disp_port_min_cap)
model.disp_port_max_cap = Param(model.PO_D, initialize = disp_port_max_cap)
#Receiving ports min and max capacities
model.rec_port_min_cap = Param(model.PO_R, initialize = rec_port_min_cap)
model.rec_port_max_cap = Param(model.PO_R, initialize = rec_port_min_cap)

##### **Parameter** - **EPA Handling Capacity**
Contains total and rail handling capacities for EPA

In [78]:
#Total and rail handling capacity for EPA
model.epa_disp_cap = Param(model.E, initialize = epa_disp_cap)
model.epa_rail_disp_cap = Param(model.E, initialize = epa_rail_disp_cap)

##### **Parameter** - **Stockyard Handling Capacity**
Contains total and rail handling capacities for Stockyard

In [79]:
#Total and rail handling capacity for stockyard
model.syd_total_thput = Param(model.S, initialize = syd_total_thput)
model.syd_rail_thput = Param(model.S, initialize = syd_rail_thput)

##### **Parameter** - **Direct and SLA Shipment**
Contains product catagory wise Direct and SLA shipment percentages for each customer node

In [80]:
#Direct Shipment Percentages
model.direct_ship_long = Param(model.C, initialize = direct_ship_long)
model.direct_ship_flat = Param(model.C, initialize = direct_ship_flat)
#Sla Shipment Percentages
model.sla_ship_long = Param(model.C, initialize = sla_ship_long)
model.sla_ship_flat = Param(model.C, initialize = sla_ship_flat)

##### **Parameter** - **Type to Form**
Contains type to form conversion rules

In [81]:
# Type to Form
model.t2f = Param(model.I, model.IM, initialize = t2f)

### **Variables**

##### **Road Transport Variables**

In [82]:
#road transport variables
##Road transport variable for Mills and EPAs pair
model.x_m_e = Var(model.M,model.E,model.IM, initialize = 0, within = NonNegativeReals,\
                  doc = 'transport quantity from mills to epa via road')
##Road transport variable for EPAs and Stockyards pair
model.x_e_s = Var(model.E,model.S,model.I, initialize = 0, within = NonNegativeReals,\
                  doc = 'transport quantity from epa to stockyard via road')
##Road transport variable for EPAs and Customers pair
model.x_e_c = Var(model.E,model.C,model.I, initialize = 0, within = NonNegativeReals,\
                  doc = 'transport quantity from epa to customers via road')
##Road transport variable for Stockyards and Stockyards pair
model.x_s_s = Var(model.S,model.S,model.I, initialize = 0, within = NonNegativeReals, \
                  doc = 'transport quantity from stockyards to stockyards via road')
##Road transport variable for Stockyards and Customers pair
model.x_s_c = Var(model.S,model.C,model.I, initialize = 0, within = NonNegativeReals, \
                  doc = 'transport quantity from stockyards to customers via road')

##### **Rail Transport Variables**

In [83]:
#rail transport variables
##Rail transport variable for Mills and EPAs pair
model.y_m_e = Var(model.M,model.E,model.IM, initialize = 0, within = NonNegativeReals, \
                  doc = 'transport quantity from mills to epa via rail')
##Rail transport variable for EPAs and Stockyards pair
model.y_e_s = Var(model.E,model.S,model.I, initialize = 0, within =NonNegativeReals, \
                  doc = 'transport quantity from epa to stockyards via rail')
##Rail transport variable for Stockyards and Stockyards pair
model.y_s_s = Var(model.S,model.S,model.I, initialize = 0, within = NonNegativeReals, \
                  doc = 'transport quantity from stockyards to stockyards via rail')

##### **Coastal Transport Variables**

In [84]:
#port transport variables
##Coastal transport variable for EPAs and Dispatch ports pair
model.z_e_po = Var(model.E, model.PO_D, model.I, initialize = 0, within = NonNegativeReals,\
                   doc = 'transport quantity from EPA to Port via road')
##Coastal transport variable for Stockyards and Dispatch ports pair
model.z_s_po = Var(model.S, model.PO_D, model.I, initialize = 0,within = NonNegativeReals,\
                   doc = 'transport quantity from SYD to Port via road')
##Coastal transport variable for Dispatch ports and Receiving ports pair
model.z_po_po= Var(model.PO_D, model.PO_R, model.I, initialize = 0,within = NonNegativeReals,\
                   doc = 'transport quantity from PORT to PORT via coast')
##Coastal transport variable for Receiving ports and customers pair
model.z_po_c = Var(model.PO_R,model.C,model.I, initialize = 0,within = NonNegativeReals,\
                   doc = 'transport quantity from port to customers via coast')

##### ***Selection Variables***

In [85]:
#EPA Selection Variable
model.p_e_select = Var(model.E, within = Binary, initialize = 0,doc = 'EPA Selection Variable')
#Stockyard Selection Variable
model.p_s_select = Var(model.S, within = Binary, initialize = 0, doc= 'Stockyard Selection Variable')

### **Objective Function**

In [86]:
def obj_rule(model):
      return sum(model.rail_cost_m_e[i,j] * sum(model.y_m_e[i,j,k] for k in model.IM) for i in model.M for j in model.E)+\
      sum(model.road_cost_m_e[i,j] * sum(model.x_m_e[i,j,k] for k in model.IM) for i in model.M for j in model.E)+\
      sum(model.rail_cost_e_s[i,j] * sum(model.y_e_s[i,j,k] for k in model.I) for i in model.E for j in model.S) +\
      sum(model.road_cost_e_s[i,j] * sum(model.x_e_s[i,j,k] for k in model.I) for i in model.E for j in model.S)+\
      sum(model.rail_cost_s_s[i,j] * sum(model.y_s_s[i,j,k] for k in model.I)for i in model.S for j in model.S)+\
      sum(model.road_cost_s_s[i,j] * sum(model.x_s_s[i,j,k] for k in model.I)for i in model.S for j in model.S)+\
      sum(model.road_cost_e_c[i,j] * sum(model.x_e_c[i,j,k] for k in model.I) for i in model.E for j in model.C)+\
      sum(model.road_cost_s_c[i,j] * sum(model.x_s_c[i,j,k] for k in model.I) for i in model.S for j in model.C)+\
      sum(model.e_port_cost[i,j] * sum(model.z_e_po[i,j,k] for k in model.I) for i in model.E for j in model.PO_D)+\
      sum(model.s_port_cost[i,j] * sum(model.z_s_po[i,j,k] for k in model.I) for i in model.S for j in model.PO_D)+\
      sum(model.port_port_cost[i,j] * sum(model.z_po_po[i,j,k] for k in model.I)for i in model.PO_D for j in model.PO_R)+\
      sum(model.port_cust_cost[i,j] * sum(model.z_po_c[i,j,k] for k in model.I)for i in model.PO_R for j in model.C)+\
      sum(model.epa_hndlng_vc_rail[i] * (sum(model.y_e_s[i,s,k] for k in model.I for s in model.S))for i in model.E)+\
      sum(model.epa_hndlng_fc_rail[i] * model.p_e_select[i] for i in model.E)+\
      sum(model.epa_hndlng_vc_road[i] * (sum(model.x_e_c[i,c,k] for c in model.C for k in model.I) +\
                                         sum(model.z_e_po[i,j,k] for j in model.PO_D for k in model.I)+\
                                         sum(model.x_e_s[i,s,k] for s in model.S for k in model.I))for i in model.E)+\
      sum(model.epa_hndlng_fc_road[i] * model.p_e_select[i] for i in model.E)+\
      sum(model.syd_hndlng_vc_rail[i] * (sum(model.y_e_s[e,i,k] for e in model.E for k in model.I)\
                                         + sum(model.y_s_s[s,i,k] for s in model.S for k in model.I)) for i in model.S)+\
      sum(model.syd_hndlng_fc_rail[i] * model.p_s_select[i] for i in model.S)+\
      sum(model.syd_hndlng_vc_road[i] * (sum(model.x_e_s[e,i,k] for e in model.E for k in model.I))for i in model.S)+\
      sum(model.syd_hndlng_fc_road[i] * model.p_s_select[i] for i in model.S)+\
      sum(model.syd_sldng_cost[i] * (sum(model.x_e_s[e,i,k] for e in model.E for k in model.I)+\
                                     sum(model.y_e_s[e,i,k] for e in model.E for k in model.I))\
          for i in model.S)+\
      sum(model.disp_port_hndlng_cost[i] * (sum(model.z_e_po[e,i,k] for e in model.E for k in model.I)+\
                                            sum(model.z_s_po[s,i,k] for s in model.S for k in model.I))\
          for i in model.PO_D)+\
      sum(model.rec_port_hndlng_cost[i] * sum(model.z_po_c[i,c,k] for c in model.C for k in model.I) for i in model.PO_R)+\
      sum(model.epa_conversn_cost[i,k] * (sum(model.x_e_s[i,s,k] for s in model.S )+\
                                          sum(model.y_e_s[i,s,k] for s in model.S )+\
                                          sum(model.x_e_c[i,c,k] for c in model.C )+\
                                          sum(model.z_e_po[i,j,k] for j in model.PO_D ))\
          for i in model.E for k in model.I)+\
      (sum((model.demand[i,k]-(((sum(model.x_s_c[s,i,k] for s in model.S)+\
                                sum(model.z_s_po[s,j,k]for s,j in zip(model.S,model.PO_D)))*\
                               model.sla_ship_flat[i])+\
                              ((sum(model.x_e_c[e,i,k] for e in model.E)+\
                                sum(model.z_e_po[e,j,k]for e,j in zip(model.E,model.PO_D)))*\
                               model.direct_ship_flat[i]))) for i,k in zip(model.C,model.I)))*9999999999\
        *model.penalty_selection

In [87]:
# Demand Penalty Choice Option
## Select 1 if you wish to include unmet demand penalty
## Select 0 otherwise
p =int(input("Enter your choice (in binary): "))
model.penalty_selection = p

Enter your choice (in binary):  1


In [88]:
model.obj = Objective(rule = obj_rule, sense = minimize)

### **Constraints**

##### **Customer Demand** Constraint

In [89]:
model.demand_constraint = ConstraintList()
## For each customer 'i', sum(flat product supplied 'k' from epa 'e' * direct shipment percentage) + \
## sum(flat product supplied 'k' from non manufacturing epa, stockyard and ports * sla shipment percentage)\
## >= (demand of product 'k' at customer node 'i')
for i in model.C:
  for k in model.FP:
    model.demand_constraint.add(expr=((sum(model.x_s_c[s,i,k] for s in model.S) +\
                                       sum(model.z_s_po[s,j,k]for s,j in zip(model.S,model.PO_D)))*\
                                       model.sla_ship_flat[i]) +\
                                     ((sum(model.x_e_c[e,i,k] for e in model.E)+\
                                       sum(model.z_e_po[e,j,k]for e,j in zip(model.E,model.PO_D)))*\
                                       model.direct_ship_flat[i])\
                                == model.demand[i,k])

In [90]:
## For each customer 'i', sum(long product supplied 'k' from epa 'e' * direct shipment percentage) + \
## sum(long product supplied 'k' from non manufacturing epa, stockyard and ports * sla shipment percentage)\
## >= (demand of product 'k' at customer node 'i')
for i in model.C:
  for k in model.LP:
    model.demand_constraint.add(expr=((sum(model.x_s_c[s,i,k] for s in model.S) +\
                                       sum(model.z_s_po[s,j,k]for s,j in zip(model.S,model.PO_D)))*\
                                       model.sla_ship_flat[i]) +\
                                     ((sum(model.x_e_c[e,i,k] for e in model.E)+\
                                       sum(model.z_e_po[e,j,k]for e,j in zip(model.E,model.PO_D)))*\
                                       model.direct_ship_flat[i])\
                                == model.demand[i,k])

#####  **Mills Supply Capacity** Constraint

In [91]:
model.m_supply_constraint = ConstraintList()
## For each mill, 'i', sum of all the product supplied, say product 'k', must be less than supply_mills
for i in model.M:
    for l in model.IM:
        model.m_supply_constraint.add(expr=(sum(model.x_m_e[i,e,l]for e in model.E)+\
                                            sum(model.y_m_e[i,e,l] for e in model.E))\
                                      <= model.supply_m[i,l])

##### **EPA Product Flow** Constraint

In [92]:
model.epa_flow_constraint = ConstraintList()
## For each EPA 'i'. sum of (product 'k' supplied from the epa 'i' * type to form conversion param[k,l])\
## == sum of (supply product 'l' from mills)
for i in model.E:
  for l in model.IM:
    model.epa_flow_constraint.add(expr =\
                                  sum(model.x_e_c[i,c,k]*model.t2f[k,l]for k in model.I for c in model.C) +\
                                  sum(model.x_e_s[i,s,k]* model.t2f[k,l]for s in model.S for k in model.I)+\
                                  sum(model.y_e_s[i,s,k]* model.t2f[k,l]for s in model.S for k in model.I) +\
                                  sum(model.z_e_po[i,j,k]* model.t2f[k,l]for j in model.PO_D for k in model.I) ==\
                                  sum(model.x_m_e[m,i,l] for m in model.M) + sum(model.y_m_e[m,i,l] for m in model.M))

##### **EPA Supply Capacity** Constraint

In [93]:
model.epa_capacity_constraint = ConstraintList()
## For each EPA 'i'. sum of (product 'k' supplied from epa 'i') 
## <= (supply capacity of product 'k' at EPA 'i' * EPA selcetion Variable)
for i in model.E:
  for k in model.I:
    model.epa_capacity_constraint.add(expr = \
                                      sum(model.x_e_c[i,c,k]*model.t2f[k,l]for k in model.I for c in model.C)+\
                                      sum(model.x_e_s[i,s,k]* model.t2f[k,l]for s in model.S for k in model.I)+\
                                      sum(model.y_e_s[i,s,k]* model.t2f[k,l]for s in model.S for k in model.I) +\
                                      sum(model.z_e_po[i,j,k]* model.t2f[k,l]for j in model.PO_D for k in model.I)\
                                      <= model.supply_epa[i,k] * model.p_e_select[i])

##### **Stockyard Product Flow** Constraint

In [94]:
model.supply_syd_constraint = ConstraintList()
## For each Stockyard, 'i', sum(product 'k' supplied from the stockyard 'i') 
## == sum(product 'k' received at the stockyard 'i')
for i in model.S:
    for k in model.I:
        model.supply_syd_constraint.add(expr=\
                                        (sum(model.x_e_s[e,i,k]for e in model.E)+\
                                         sum(model.y_e_s[e,i,k]for e in model.E)+\
                                         sum(model.x_s_s[s,i,k]for s in model.S)+\
                                         sum(model.y_s_s[s,i,k]for s in model.S))==\
                                        (sum(model.z_s_po[i,j,k]for j in model.PO_D)+\
                                          sum(model.x_s_s[i,s,k]for s in model.S)+\
                                          sum(model.y_s_s[i,s,k]for s in model.S)+\
                                          sum(model.x_s_c[i,c,k] for c in model.C)))

##### **Stockyard Supply Capacity** Constraint

In [95]:
model.syd_capacity_constraint = ConstraintList()
## For each Stockyard 'i'. sum(product supplied 'k')<= (Handling capacity of product 'k' at stockyard 'i')
for i in model.S:
  model.syd_capacity_constraint.add(expr=\
                                    (sum(model.z_s_po[i,j,k] for j in model.PO_D)+\
                                     sum(model.x_s_s[i,s,k] for s in model.S)+\
                                     sum(model.y_s_s[i,s,k]for s in model.S)+\
                                     sum(model.x_s_c[i,c,k] for c in model.C))\
                                    <= model.syd_total_thput[i] *model.p_s_select[i])
  model.syd_capacity_constraint.add(expr=\
                                    sum(model.y_s_s[i,s,k] for s in model.S for k in model.I)\
                                    <= model.syd_rail_thput[i] * model.p_s_select[i])

##### **Port Product Flow** Constraint

In [96]:
model.port_flow_constraint = ConstraintList()
## for each dispatch port 'i', sum(product 'k' supplied from the dispatch port 'i') \
## == sum(product 'k' received at the dispatch port 'i')
for i in model.PO_D:
  for k in model.I:
    model.port_flow_constraint.add(expr=\
                                   (sum(model.z_e_po[e,i,k] for e in model.E)+\
                                    sum(model.z_s_po[s,i,k]for s in model.S)) == \
                                    sum(model.z_po_po[i,j,k]for j in model.PO_R))

## for each receiving port 'i', sum(product 'k' supplied from the receiving port 'i') \
## == sum(product 'k' received at the receiving port 'i')
for i in model.PO_R:
  for k in model.I:
    model.port_flow_constraint.add(expr=\
                                   (sum(model.z_po_po[j,i,k]for j in model.PO_D))\
                                    == sum(model.z_po_c[i,c,k] for c in model.C))

##### **Port Handling Capacity** Constraint

In [97]:
model.disp_port_hndlng_cap = ConstraintList()
## For each dispatch Port 'i'. sum of (product supplied 'k') \
## <= (Handling capacity of product 'k' at Port 'i')
for i in model.PO_D:
  model.disp_port_hndlng_cap.add(inequality(model.disp_port_min_cap[i],\
                                            sum(model.z_po_po[i,j,k] for j in model.PO_R for k in model.I),\
                                            model.disp_port_max_cap[i]))
model.rec_port_hndlng_cap = ConstraintList()
## For each receiving Port 'i'. sum of (product supplied 'k') \
## <= (Handling capacity of product 'k' at Port 'i')
for i in model.PO_R:
  model.rec_port_hndlng_cap.add(inequality(model.rec_port_min_cap[i],\
                                           sum(model.z_po_c[i,c,k]for c in model.C for k in model.I),\
                                           model.rec_port_max_cap[i]))

# **SOLUTION**

In [98]:
result_cost = SolverFactory('glpk').solve(model)
result_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 48216856557.9922
  Upper bound: 48216856557.9922
  Number of objectives: 1
  Number of constraints: 25183
  Number of variables: 3769225
  Number of nonzeros: 9374007
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 621.1929180622101
# ----------------------------------------------------------
#   Solution Information
# -

# **EXPORTING SOLUTION TO EXCEL**

##### Reading output variable values as list

In [99]:
# Defining a function to read variable values into python lists
## Inputs to the function are as follows-
### frst_itr - first index of the variable(From)
### scnd_itr - second index of the variable(To)
### thrd_itr - third index of the variable(Products)
def lst(frst_itr, scnd_itr,mode, thrd_itr, var):
  p = []
  for i in range(0,len(frst_itr)):
    for j in range(0, len(scnd_itr)):
      for k in range(0, len(thrd_itr)):
        x = frst_itr[i]
        y = scnd_itr[j]
        z = thrd_itr[k]
        if var[x,y,z] !=0:
          p.append([x, y,mode, z , var[x,y,z].value])
          k+=1
        else:
          k+=1
      j+=1
    i+=1
  return p

In [100]:
# Reading output road variables as lists 
var_m_e_road = lst(mills, epa,'Road', product_catagories, model.x_m_e)
var_e_s_road = lst(epa,syd,'Road',end_products,model.x_e_s)
var_e_c = lst(epa,customers,'Road',end_products, model.x_e_c)
var_s_s_road = lst(syd, syd,'Road', end_products, model.x_s_s)
var_s_c = lst(syd,customers,'Road', end_products, model.x_s_c)
print(len(var_m_e_road),len(var_e_s_road),len(var_e_c),len(var_s_s_road),\
      len(var_s_c))

24 67 579 50 1990


In [101]:
# Reading output rail variables as lists 
var_m_e_rail = lst(mills, epa,'Rail', product_catagories, model.y_m_e)
var_e_s_rail = lst(epa,syd,'Rail',end_products,model.y_e_s)
var_s_s_rail = lst(syd, syd,'Rail', end_products, model.y_s_s)
print(len(var_m_e_rail),len(var_e_s_rail),len(var_s_s_rail))

39 300 0


In [102]:
# Reading output coastal variables as lists 
var_e_po = lst(epa, disp_ports,'Coastal', end_products, model.z_e_po)
var_s_po = lst(syd, disp_ports,'Coastal', end_products, model.z_s_po)
var_po_po = lst(disp_ports, rec_ports,'Coastal',end_products, model.z_po_po)
var_po_c = lst(rec_ports, customers,'Coastal', end_products, model.z_po_c)
print(len(var_e_po),len(var_s_po),len(var_po_po),len(var_po_c))

1 0 5 5


In [103]:
# Reading output selection variables as lists 
var_e_select = list(['%s' %i, '%d' %model.p_e_select[i].value] for i in model.E)
var_s_select = list(['%s' %i, '%d' %model.p_s_select[i].value] for i in model.S)
print(len(var_e_select),len(var_s_select))

76 76


##### Converting Lists into DataFrame and transferring to Excel

In [104]:
# Creating Excel Output file
book = opxl.load_workbook('Output.xlsx')
writer = pd.ExcelWriter('Output.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
writer.save()
# Defining a function to covert lists to dataftame
def to_df(list_of_lists):
    x = pd.DataFrame(columns = ['From', 'To','Mode', 'Product', 'Quantity'])
    for i in list_of_lists:
        x = x.append(pd.DataFrame(i, columns = ['From', 'To','Mode', 'Product', 'Quantity']))
    return x

In [105]:
# Combining all  transportation variables into DataFrame
cust_variables = [var_e_c, var_s_c, var_po_c]
cust_variables_df = to_df(cust_variables)
cust_variables_df.to_excel(writer, sheet_name='Customer')
writer.save()

In [106]:
# Representing the customer demand met by the transportation model
cust_variables_df.iloc[:,4] = cust_variables_df.iloc[:,4].astype(float)
demand_met = pd.pivot_table(cust_variables_df,index=cust_variables_df['To'],\
                            columns=cust_variables_df['Product'],\
                           aggfunc=[np.sum],fill_value=0)
demand_met.to_excel(writer, sheet_name='Demand_met')
writer.save()

In [107]:
# Combining all coastal output variables in dataframe
coastal_var = [var_po_po]
coast_var_df = to_df(coastal_var)
coast_var_df.to_excel(writer, sheet_name='Coast')
writer.save()

In [108]:
# Combining all mills output variables in dataframe
mills_epa = [var_m_e_road, var_m_e_rail]
mills_epa_df = to_df(mills_epa)
mills_epa_df.to_excel(writer, sheet_name='Mills')
writer.save()

In [109]:
# Combining all epa output variables in dataframe
epa_syd = [var_e_s_road,var_e_s_rail, var_e_po]
epa_syd_df = to_df(epa_syd)
epa_syd_df.to_excel(writer, sheet_name='EPA')
writer.save()

In [110]:
# Combining all stockyard output variables in dataframe
syd_syd = [var_s_s_road,var_s_s_rail,var_s_po]
syd_syd_df = to_df(syd_syd)
syd_syd_df.to_excel(writer, sheet_name='Stockyard')
writer.save()

In [ ]:
# Output value of total transportation cost
model.transportation_cost=Param(initialize=\
(sum(model.rail_cost_m_e[i,j] * sum(model.y_m_e[i,j,k] for k in model.IM) for i in model.M for j in model.E)+\
 sum(model.road_cost_m_e[i,j] * sum(model.x_m_e[i,j,k] for k in model.IM) for i in model.M for j in model.E)+\
 sum(model.rail_cost_e_s[i,j] * sum(model.y_e_s[i,j,k] for k in model.I) for i in model.E for j in model.S) +\
 sum(model.road_cost_e_s[i,j] * sum(model.x_e_s[i,j,k] for k in model.I) for i in model.E for j in model.S)+\
 sum(model.rail_cost_s_s[i,j] * sum(model.y_s_s[i,j,k] for k in model.I)for i in model.S for j in model.S)+\
 sum(model.road_cost_s_s[i,j] * sum(model.x_s_s[i,j,k] for k in model.I)for i in model.S for j in model.S)+\
 sum(model.road_cost_e_c[i,j] * sum(model.x_e_c[i,j,k] for k in model.I) for i in model.E for j in model.C)+\
 sum(model.road_cost_s_c[i,j] * sum(model.x_s_c[i,j,k] for k in model.I) for i in model.S for j in model.C)+\
 sum(model.e_port_cost[i,j] * sum(model.z_e_po[i,j,k] for k in model.I) for i in model.E for j in model.PO_D)+\
 sum(model.s_port_cost[i,j] * sum(model.z_s_po[i,j,k] for k in model.I) for i in model.S for j in model.PO_D)+\
 sum(model.port_port_cost[i,j] * sum(model.z_po_po[i,j,k] for k in model.I)for i in model.PO_D for j in model.PO_R)+\
 sum(model.port_cust_cost[i,j] * sum(model.z_po_c[i,j,k] for k in model.I)for i in model.PO_R for j in model.C)))
trans_cost = ['Transportation Cost', model.transportation_cost.value]

In [ ]:
# Output value of epa handling cost
model.epa_handling_cost=Param(initialize=\
(sum(model.epa_hndlng_vc_rail[i] * (sum(model.y_e_s[i,s,k] for k in model.I for s in model.S))for i in model.E)+\
 sum(model.epa_hndlng_fc_rail[i] * model.p_e_select[i] for i in model.E)+\
 sum(model.epa_hndlng_vc_road[i] * (sum(model.x_e_c[i,c,k] for c in model.C for k in model.I) +\
                                 sum(model.z_e_po[i,j,k] for j in model.PO_D for k in model.I)+\
                                 sum(model.x_e_s[i,s,k] for s in model.S for k in model.I))for i in model.E)+\
 sum(model.epa_hndlng_fc_road[i] * model.p_e_select[i] for i in model.E)+\
 sum(model.epa_hndlng_vc_rail[i] * (sum(model.y_e_s[i,s,k] for k in model.I for s in model.S))for i in model.E)+\
 sum(model.epa_hndlng_fc_rail[i] * model.p_e_select[i] for i in model.E)+\
 sum(model.epa_hndlng_vc_road[i] * (sum(model.x_e_c[i,c,k] for c in model.C for k in model.I) +\
                                 sum(model.z_e_po[i,j,k] for j in model.PO_D for k in model.I)+\
                                 sum(model.x_e_s[i,s,k] for s in model.S for k in model.I))for i in model.E)+\
 sum(model.epa_hndlng_fc_road[i] * model.p_e_select[i] for i in model.E)))
epa_handling = ['EPA Handling Costs', model.epa_handling_cost.value]

In [ ]:
# Output value of stockyard handling cost
model.syd_handling_cost=Param(initialize=\
(sum(model.syd_hndlng_vc_rail[i] * (sum(model.y_e_s[e,i,k] for e in model.E for k in model.I)\
                                 + sum(model.y_s_s[s,i,k] for s in model.S for k in model.I)) for i in model.S)+\
 sum(model.syd_hndlng_fc_rail[i] * model.p_s_select[i] for i in model.S)+\
 sum(model.syd_hndlng_vc_road[i] * (sum(model.x_e_s[e,i,k] for e in model.E for k in model.I))for i in model.S)+\
 sum(model.syd_hndlng_fc_road[i] * model.p_s_select[i] for i in model.S)+\
 sum(model.syd_sldng_cost[i] * (sum(model.x_e_s[e,i,k] for e in model.E for k in model.I)+\
                             sum(model.y_e_s[e,i,k] for e in model.E for k in model.I))\
  for i in model.S)))
syd_handling = ['Stockyard Handling Costs', model.syd_handling_cost.value]

In [ ]:
# Output value of ports handling cost
model.ports_handling_cost=Param(initialize=\
(sum(model.disp_port_hndlng_cost[i] * (sum(model.z_e_po[e,i,k] for e in model.E for k in model.I)+\
                                    sum(model.z_s_po[s,i,k] for s in model.S for k in model.I))\
  for i in model.PO_D)+\
 sum(model.rec_port_hndlng_cost[i] * sum(model.z_po_c[i,c,k] for c in model.C for k in model.I) for i in model.PO_R)))
port_handling = ['Ports Handling Costs', model.ports_handling_cost.value]

In [ ]:
# Output value of product conversion cost
model.product_conversion_cost=Param(initialize=\
(sum(model.epa_conversn_cost[i,k] * (sum(model.x_e_s[i,s,k] for s in model.S )+\
                                  sum(model.y_e_s[i,s,k] for s in model.S )+\
                                  sum(model.x_e_c[i,c,k] for c in model.C )+\
                                  sum(model.z_e_po[i,j,k] for j in model.PO_D ))\
  for i in model.E for k in model.I)))
product_cnvrsn = ['Product Conversion Costs', model.product_conversion_cost.value]

In [ ]:
# Percentage value of unmet demand
model.unmet_demand_percent=Param(initialize=\
(sum((model.demand[i,k]-(((sum(model.x_s_c[s,i,k] for s in model.S)+\
                        sum(model.z_s_po[s,j,k]for s,j in zip(model.S,model.PO_D)))*\
                       model.sla_ship_flat[i])+\
                      ((sum(model.x_e_c[e,i,k] for e in model.E)+\
                        sum(model.z_e_po[e,j,k]for e,j in zip(model.E,model.PO_D)))*\
                       model.direct_ship_flat[i]))) for i,k in zip(model.C,model.I))/\
 sum(model.demand[i,k]for i,k in zip(model.C,model.I)))*100)
unmet_demand = ['Unmet Demand Percentage (%)', model.unmet_demand_percent.value]

In [ ]:
# Combining all combining all the costs occured in transporation into dataframe
costs = pd.DataFrame([trans_cost,epa_handling,syd_handling,product_cnvrsn,unmet_demand],\
                    columns = ['Parameter','Value'])
costs.to_excel(writer, sheet_name='Costs')
writer.save()

In [ ]:
import folium as fp